In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np

np.random.seed(42)

accounts = ['ACC123456', 'ACC234567', 'ACC345678', 'ACC456789', 'ACC567890']
merchants = ['ABC Store', 'ATM', 'Employer', 'Stock Exchange', 'XYZ Electronics', 'Online Store', 'Cafe', 'Supermarket']

n_rows = 10000

data = {
    'Transaction ID': np.arange(1000000000, 1000000000 + n_rows),
    'Account ID': np.random.choice(accounts, n_rows),
    'Transaction Amount': np.round(np.random.uniform(10.00, 2000.00, n_rows), 2),
    'Transaction Type': np.random.choice(['Purchase', 'Withdrawal', 'Deposit', 'Trade'], n_rows),
    'Merchant/Counterparty': np.random.choice(merchants, n_rows),
    'Location': np.random.choice(['New York, NY', 'Los Angeles, CA', 'Chicago, IL', 'Miami, FL', 'Online'], n_rows),
    'Date and Time': pd.date_range(start='2024-08-01', periods=n_rows, freq='H').strftime('%Y-%m-%d %H:%M').tolist()
}

pdf = pd.DataFrame(data)
parquet_file = "/content/drive/MyDrive/Lecture13/encrypted_financial.parquet"

<ipython-input-4-b0d47072fdd1>:18: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  'Date and Time': pd.date_range(start='2024-08-01', periods=n_rows, freq='H').strftime('%Y-%m-%d %H:%M').tolist()


In [5]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
        .appName("NativeParquetEncryption")
        .config("spark.hadoop.parquet.encryption.kms.client.class",
                "org.apache.parquet.crypto.keytools.mocks.InMemoryKMS")
        .config("spark.hadoop.parquet.encryption.key.list",
                "keyA:AAECAwQFBgcICQoLDA0ODw==,keyB:AQIDBAUGBwgJCgsMDQ4PAA==")
        .getOrCreate()
)

df = spark.createDataFrame(pdf)

df.write \
  .option("parquet.encryption.column.keys", "keyA:Account ID,Location") \
  .option("parquet.encryption.footer.key",    "keyB") \
  .mode("overwrite") \
  .parquet(parquet_file)

In [6]:
from pyspark.sql.functions import col, sum as spark_sum, max as spark_max

df2 = spark.read.parquet(parquet_file)

# Group by 'Account ID' and 'Merchant/Counterparty' and sum 'Transaction Amount'
grouped_df = df.groupBy('Account ID', 'Merchant/Counterparty').agg(spark_sum('Transaction Amount').alias('Total Spending'))

# Find the maximum 'Total Spending' for each 'Account ID'
max_spending_df = grouped_df.groupBy('Account ID').agg(spark_max('Total Spending').alias('Max Spending'))

highest_spending_df = grouped_df.join(max_spending_df,
                                      on=['Account ID'],
                                      how='inner') \
                                .filter(col('Total Spending') == col('Max Spending'))

highest_spending_df = highest_spending_df.drop('Max Spending')
highest_spending_df.show(truncate=False)


+----------+---------------------+------------------+
|Account ID|Merchant/Counterparty|Total Spending    |
+----------+---------------------+------------------+
|ACC345678 |Cafe                 |271944.0800000001 |
|ACC123456 |Stock Exchange       |279712.86999999994|
|ACC456789 |XYZ Electronics      |266071.95         |
|ACC234567 |XYZ Electronics      |278558.41000000003|
|ACC567890 |Employer             |291354.99999999994|
+----------+---------------------+------------------+

